In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['font.serif'] = ['SimHei']

import seaborn as sns
%matplotlib inline
p = sns.color_palette()
sns.set_style("darkgrid",{"font.sans-serif":['simhei', 'Arial']})

# 天气数据

weather_data = pd.read_table('/Users/tuyu/000000毕业设计/航班动态算法大赛原始数据/train/weatherDataall.csv', sep=',', encoding='utf-8',low_memory=False)
weather_data.head()

,城市,天气,最低气温,最高气温,日期,Unnamed: 5
0,厦门,阴,21,26,2015-05-01,NaN
1,宁德,阵雨,19,25,2015-05-01,NaN
2,莆田,阵雨转阴,20,26,2015-05-01,NaN
3,泉州,阴,22,26,2015-05-01,NaN
4,漳州,阵雨转阴,22,28,2015-05-01,NaN


In [2]:
del(weather_data['Unnamed: 5'])
#读取爬下来的数据


weatherData2015 = pd.read_excel('/Users/tuyu/000000毕业设计/航班动态算法大赛原始数据/weatherData2015.xlsx',encoding='utf-8')
weatherData2015 = weatherData2015.rename(columns={'城市名称':'城市','ymd':'日期','bWendu':'最高气温','yWendu':'最低气温','tianqi':'天气','fengxiang':'风向','fengli':'风力'})

In [3]:
cityData = pd.read_excel('/Users/tuyu/000000毕业设计/航班动态算法大赛原始数据/train/cityData.xlsx',enconding='utf-8')
cityData.rename(columns={'城市名称':'city'})
label = cityData.ix[:,-1]
loc = label.tolist()
# print(loc)
#去除不在列表里的城市
weather_data[weather_data.城市.isin(loc)]

/Users/tuyu/Library/Python/3.6/lib/python/site-packages/ipykernel_launcher.py:3: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


,城市,天气,最低气温,最高气温,日期
0,厦门,阴,21,26,2015-05-01
5,龙岩,小雨转阴,20,26,2015-05-01
6,三明,小雨,20,25,2015-05-01
8,南昌,小雨转中雨,22,27,2015-05-01
9,九江,雷阵雨转大雨,19,28,2015-05-01
...,...,...,...,...,...
827483,宁蒗,多云,14,NaN,2017-05-31
827515,长白山,小雨,7,NaN,2017-05-31
827555,梅县,多云,24,NaN,2017-05-31
827572,满洲里,晴,-2,NaN,2017-05-31


In [4]:
# 将datetime类型转换为object类型
weatherData2015['日期'] = weatherData2015['日期'].apply(lambda x:x.strftime('%Y-%m-%d') if not(pd.isnull(x)) else None)
weatherData2015.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121863 entries, 0 to 121862
Data columns (total 7 columns):
城市      121863 non-null object
日期      121863 non-null object
最高气温    121863 non-null int64
最低气温    121863 non-null int64
天气      121863 non-null object
风向      121863 non-null object
风力      121861 non-null object
dtypes: int64(2), object(5)
memory usage: 6.5+ MB


In [5]:
#将城市和日期组合成一列，然后合并数据
def my_point(a,b):
    return (a,b)
weather_data['point'] = weather_data.apply(lambda row:my_point(row['城市'],row['日期']), axis=1)
weatherData2015['point'] = weatherData2015.apply(lambda row:my_point(row['城市'],row['日期']), axis=1)
weather_data = pd.merge(weather_data, weatherData2015, how = 'left', on='point')
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 828469 entries, 0 to 828468
Data columns (total 13 columns):
城市_x      828469 non-null object
天气_x      828399 non-null object
最低气温_x    778400 non-null object
最高气温_x    678990 non-null object
日期_x      828469 non-null object
point     828469 non-null object
城市_y      74953 non-null object
日期_y      74953 non-null object
最高气温_y    74953 non-null float64
最低气温_y    74953 non-null float64
天气_y      74953 non-null object
风向        74953 non-null object
风力        74953 non-null object
dtypes: float64(2), object(11)
memory usage: 88.5+ MB


In [6]:
weather_data = weather_data.rename(columns={'城市_x':'城市','天气_x':'天气','最低气温_x':'最低气温','最高气温_x':'最高气温','日期_x':'日期'})

In [7]:

#去除不在列表里的城市
weather_data = weather_data[weather_data.城市.isin(loc)]
weather_data = weather_data.drop(['point', '城市_y', '日期_y',
       '最高气温_y', '最低气温_y', '天气_y'], axis=1)

In [8]:
open_day='2015-05-01'
close_day='2016-01-01'
con1=weather_data['日期']>=open_day
con2=weather_data['日期']<close_day
weather2015=weather_data[con1&con2]

In [9]:
weatherData2016 = pd.read_excel('/Users/tuyu/000000毕业设计/航班动态算法大赛原始数据/weatherData2016.xlsx',encoding='utf-8')

In [10]:
open_day='2016-01-01'
close_day='2017-01-01'
con1=weather_data['日期']>=open_day
con2=weather_data['日期']<close_day
weather2016=weather_data[con1&con2]

In [11]:
weatherData2016['ymd'] = weatherData2016['ymd'].astype(str)
weatherData2016['ymd'] = weatherData2016['ymd'].apply(lambda x:x[0:10] if not(pd.isnull(x)) else None)
weatherData2016.head()

,aqi,aqiInfo,aqiLevel,bWendu,fengli,fengxiang,tianqi,yWendu,ymd,城市名称
0,NaN,NaN,NaN,24,3-4级,东北风,多云,18,2016-01-01,东方
1,NaN,NaN,NaN,26,3-4级,东北风,多云,18,2016-01-02,东方
2,NaN,NaN,NaN,26,3-4级,东风,多云,20,2016-01-03,东方
3,NaN,NaN,NaN,27,3-4级,东南风,多云,21,2016-01-04,东方
4,NaN,NaN,NaN,25,微风,西南风,阴~小雨,19,2016-01-05,东方


In [12]:
weatherData2016['point'] = weatherData2016.apply(lambda row:my_point(row['城市名称'],row['ymd']), axis=1)
weather2016['point'] = weather2016.apply(lambda row:my_point(row['城市'],row['日期']), axis=1)
weather2016 = pd.merge(weather2016, weatherData2016, how = 'left', on='point')
weather2016.head()

/Users/tuyu/Library/Python/3.6/lib/python/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,城市,天气,最低气温,最高气温,日期,风向,风力,point,aqi,aqiInfo,aqiLevel,bWendu,fengli,fengxiang,tianqi,yWendu,ymd,城市名称
0,重庆,阴,9,13,2016-01-01,NaN,NaN,"(重庆, 2016-01-01)",201.0,重度污染,5.0,13.0,微风,无持续风向,阴,10.0,2016-01-01,重庆
1,哈尔滨,阵雪,-19,-5,2016-01-01,NaN,NaN,"(哈尔滨, 2016-01-01)",205.0,重度污染,5.0,-5.0,微风,东风,霾~多云,-20.0,2016-01-01,哈尔滨
2,齐齐哈尔,晴,-22,-10,2016-01-01,NaN,NaN,"(齐齐哈尔, 2016-01-01)",109.0,轻度污染,3.0,-10.0,微风,北风,晴~多云,-21.0,2016-01-01,齐齐哈尔
3,牡丹江,多云转阵雪,-11,-5,2016-01-01,NaN,NaN,"(牡丹江, 2016-01-01)",154.0,中度污染,4.0,-5.0,4-5级,西风,多云~阵雪,-11.0,2016-01-01,牡丹江
4,佳木斯,阵雪,-17,-9,2016-01-01,NaN,NaN,"(佳木斯, 2016-01-01)",92.0,良,2.0,-9.0,3-4级,南风,阵雪,-19.0,2016-01-01,佳木斯


In [13]:
weather2016.drop(['风向', '风力','point', 'bWendu', 'tianqi', 'yWendu', 'ymd',
       '城市名称'],axis=1,inplace=True)
weather2016 = weather2016.rename(columns={'fengli':'风力','fengxiang':'风向'})

In [14]:
weatherData2017 = pd.read_excel('/Users/tuyu/000000毕业设计/航班动态算法大赛原始数据/weatherData2017.xlsx',encoding='utf-8')

In [15]:
open_day='2017-01-01'
close_day='2018-01-01'
con1=weather_data['日期']>=open_day
con2=weather_data['日期']<close_day
weather2017=weather_data[con1&con2]

In [16]:
weatherData2017['ymd'] = weatherData2017['ymd'].astype(str)
weatherData2017['ymd'] = weatherData2017['ymd'].apply(lambda x:x[0:10] if not(pd.isnull(x)) else None)
weatherData2017.head()

,aqi,aqiInfo,aqiLevel,bWendu,fengli,fengxiang,tianqi,yWendu,ymd,城市名称
0,118.0,轻度污染,3.0,22,微风,无持续风向,多云~晴,17,2017-01-01,江门
1,142.0,轻度污染,3.0,25,微风,无持续风向,多云~晴,18,2017-01-02,江门
2,180.0,中度污染,4.0,26,微风,无持续风向,晴,17,2017-01-03,江门
3,85.0,良,2.0,26,微风,无持续风向,多云,17,2017-01-04,江门
4,132.0,轻度污染,3.0,26,微风,无持续风向,多云,19,2017-01-05,江门


In [17]:
weatherData2017['point'] = weatherData2017.apply(lambda row:my_point(row['城市名称'],row['ymd']), axis=1)
weather2017['point'] = weather2017.apply(lambda row:my_point(row['城市'],row['日期']), axis=1)
weather2017 = pd.merge(weather2017, weatherData2017, how = 'left', on='point')

/Users/tuyu/Library/Python/3.6/lib/python/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [18]:
weather2017.drop([ '风向', '风力', 'point', 'bWendu','tianqi', 'yWendu', 'ymd','城市名称'],axis=1,inplace=True)
weather2017 = weather2017.rename(columns={'fengli':'风力','fengxiang':'风向'})

In [19]:
weather2016_2017 = pd.concat([weather2016, weather2017], axis=0, ignore_index=True)
weather2016_2017

,城市,天气,最低气温,最高气温,日期,aqi,aqiInfo,aqiLevel,风力,风向
0,重庆,阴,9,13,2016-01-01,201.0,重度污染,5.0,微风,无持续风向
1,哈尔滨,阵雪,-19,-5,2016-01-01,205.0,重度污染,5.0,微风,东风
2,齐齐哈尔,晴,-22,-10,2016-01-01,109.0,轻度污染,3.0,微风,北风
3,牡丹江,多云转阵雪,-11,-5,2016-01-01,154.0,中度污染,4.0,4-5级,西风
4,佳木斯,阵雪,-17,-9,2016-01-01,92.0,良,2.0,3-4级,南风
...,...,...,...,...,...,...,...,...,...,...
100293,宁蒗,多云,14,NaN,2017-05-31,NaN,NaN,NaN,NaN,NaN
100294,长白山,小雨,7,NaN,2017-05-31,NaN,NaN,NaN,NaN,NaN
100295,梅县,多云,24,NaN,2017-05-31,NaN,NaN,NaN,NaN,NaN
100296,满洲里,晴,-2,NaN,2017-05-31,NaN,NaN,NaN,NaN,NaN


In [20]:
Flight_SpecInfoData = pd.read_table('/Users/tuyu/000000毕业设计/Flight_Delay/Flight_SpecInfoDataFeature.csv', sep=',',encoding='gb2312')


In [21]:
open_day='2016-01-01'
close_day='2018-01-01'
con1=Flight_SpecInfoData['计划起飞日期']>=open_day
con2=Flight_SpecInfoData['计划起飞日期']<close_day
Flight_SpecInfoData2016=Flight_SpecInfoData[con1&con2]

In [22]:
city_data = pd.read_excel('/Users/tuyu/000000毕业设计/航班动态算法大赛原始数据/train/cityData.xlsx')


In [23]:
weather2016 = pd.merge(city_data, weather2016, how='left',left_on='城市名称',right_on='城市')
del(weather2016['城市名称'])
del(weather2016['城市'])

In [24]:
Flight_SpecInfoData2016 = pd.merge(Flight_SpecInfoData2016, weather2016, how='left',left_on=['出发机场','计划起飞日期'],right_on=['机场编码','日期'])
Flight_SpecInfoData2016 = Flight_SpecInfoData2016.rename(columns={'天气':'出发机场天气','最低气温':'出发机场最低气温',
                                                                  '最高气温':'出发机场最高气温','风向':'出发机场风向','风力':'出发机场风力',
                                                                 'aqi':'出发机场aqi','aqiInfo':'出发机场aqiInfo','aqiLevel':'出发机场aqiLevel'})
del(Flight_SpecInfoData2016['机场编码'])
del(Flight_SpecInfoData2016['日期'])

In [25]:
Flight_SpecInfoData2016 = pd.merge(Flight_SpecInfoData2016, weather2016, how='left',left_on=['到达机场','计划起飞日期'],right_on=['机场编码','日期'])

Flight_SpecInfoData2016 = Flight_SpecInfoData2016.rename(columns={'天气':'到达机场天气','最低气温':'到达机场最低气温',
                                                                  '最高气温':'到达机场最高气温','风向':'到达机场风向','风力':'到达机场风力',
                                                                 'aqi':'到达机场aqi','aqiInfo':'到达机场aqiInfo','aqiLevel':'到达机场aqiLevel'})
del(Flight_SpecInfoData2016['机场编码'])
del(Flight_SpecInfoData2016['日期'])

In [26]:
All_Data = Flight_SpecInfoData2016
#构建标签
All_Data['是否延误3小时以上'] = All_Data['起飞延误时间'].apply(lambda x : 0 if x < 3 else 1)


In [27]:
print(sum(All_Data['出发机场风力'].isnull()), sum(All_Data['出发机场天气'].isnull()),
      sum(All_Data['出发机场最低气温'].isnull()),sum(All_Data['出发机场最高气温'].isnull()),
     sum(All_Data['出发机场aqi'].isnull()),sum(All_Data['出发机场aqiInfo'].isnull()))

1764010 1650991 1652292 1657967 1767801 1767801


In [28]:
print(sum(All_Data['到达机场风力'].isnull()), sum(All_Data['到达机场天气'].isnull()),
      sum(All_Data['到达机场最低气温'].isnull()),sum(All_Data['到达机场最高气温'].isnull()),
     sum(All_Data['到达机场aqi'].isnull()),sum(All_Data['到达机场aqiInfo'].isnull()))

1757475 1649078 1650380 1656021 1761226 1761226


In [29]:
print(sum(All_Data['出发机场风力'].isnull() & All_Data['到达机场风力'].isnull()), 
      sum(All_Data['出发机场天气'].isnull() & All_Data['到达机场天气'].isnull()),
      sum(All_Data['出发机场最低气温'].isnull() & All_Data['到达机场最低气温'].isnull()),
      sum(All_Data['出发机场最高气温'].isnull() & All_Data['到达机场最高气温'].isnull()),
      sum(All_Data['出发机场aqi'].isnull() & All_Data['到达机场aqi'].isnull()),
      sum(All_Data['出发机场aqiInfo'].isnull() & All_Data['到达机场aqiInfo'].isnull())
     )

1609548 1604904 1604904 1605004 1609548 1609548


In [30]:
print(sum(All_Data['出发机场风力'].isnull() & All_Data['是否延误3小时以上'] ==1), 
      sum(All_Data['出发机场天气'].isnull() & All_Data['是否延误3小时以上'] ==1),
      sum(All_Data['出发机场最低气温'].isnull() & All_Data['是否延误3小时以上'] ==1),
      sum(All_Data['出发机场最高气温'].isnull() & All_Data['是否延误3小时以上'] ==1),
      sum(All_Data['出发机场aqi'].isnull() & All_Data['是否延误3小时以上'] ==1),
      sum(All_Data['出发机场aqiInfo'].isnull() & All_Data['是否延误3小时以上'] ==1)
     )
print(sum(All_Data['是否延误3小时以上'] ==1) / len(All_Data['是否延误3小时以上']))
print(sum(All_Data['是否延误3小时以上'] ==1))

114328 106205 106398 106777 114749 114749
0.06517742186451227
341805


In [31]:
#对出发机场风力等级进行评分
#空值表示为0
All_Data['出发机场风力等级评分'] = All_Data['出发机场风力'].apply(lambda x : 
                                                   10 if x == '微风' else ( 9 if x == '1-2级' else (8 if x == '3-4级' 
                                                                    else (7 if x == '4-5级' else (6 if x == '5-6级' 
                                                                    else (5 if x == '6-7级' else (4 if x == '7-8级' 
                                                                 else (3 if x == '8-9级' else (2 if x == '9-10级' 
                                                                else (1 if x == '10-11级' else 0 ))))))))))

#对到达机场风力等级进行评分
#空值表示为0
All_Data['到达机场风力等级评分'] = All_Data['到达机场风力'].apply(lambda x : 
                                                   10 if x == '微风' else ( 9 if x == '1-2级' else (8 if x == '3-4级' 
                                                                    else (7 if x == '4-5级' else (6 if x == '5-6级' 
                                                                    else (5 if x == '6-7级' else (4 if x == '7-8级' 
                                                                 else (3 if x == '8-9级' else (2 if x == '9-10级' 
                                                                else (1 if x == '10-11级' else 0 ))))))))))

In [32]:
#出发机场aqi等级评分
#空值表示为0
All_Data['出发机场aqi等级评分'] = All_Data['出发机场aqiInfo'].apply(lambda x : 6 if x == '优' else (5 if x == '良' 
                                                        else (4 if x == '轻度污染' else (3 if x == '中度污染' 
                                                        else(2 if x == '重度污染' 
                                                        else (1 if x == '严重污染' else 0))))))

#到达机场aqi等级评分
#空值表示为0
All_Data['到达机场aqi等级评分'] = All_Data['到达机场aqiInfo'].apply(lambda x : 6 if x == '优' else (5 if x == '良' 
                                                        else (4 if x == '轻度污染' else (3 if x == '中度污染' 
                                                        else(2 if x == '重度污染' 
                                                        else (1 if x == '严重污染' else 0))))))

In [33]:
WeatherScore = pd.read_excel('/Users/tuyu/000000毕业设计/WeatherScore.xlsx')
All_Data = pd.merge(All_Data, WeatherScore, how='left',left_on='出发机场天气', right_on='天气')
All_Data.head()

,出发机场,到达机场,航班编号,计划起飞时间,计划到达时间,实际起飞时间,实际到达时间,飞机编号,航班是否取消,计划起飞时间1,...,到达机场aqiLevel,到达机场风力,到达机场风向,是否延误3小时以上,出发机场风力等级评分,到达机场风力等级评分,出发机场aqi等级评分,到达机场aqi等级评分,天气,得分
0,SWA,TAO,CZ3855,1451606400,1451619600,NaN,NaN,0.0,取消,2016-01-01 00:00:00+00:00,...,3.0,4-5级,西南风,1,10,7,4,4,多云,8.0
1,NNG,CAN,CZ3295,1451606400,1451610600,NaN,NaN,0.0,取消,2016-01-01 00:00:00+00:00,...,2.0,微风,无持续风向,1,10,10,5,5,多云,8.0
2,SWA,NGB,CZ3855,1451606400,1451610900,NaN,NaN,0.0,取消,2016-01-01 00:00:00+00:00,...,2.0,微风,东南风,1,10,10,4,5,多云,8.0
3,CSX,TNA,CZ3983,1451606400,1451613600,NaN,NaN,0.0,取消,2016-01-01 00:00:00+00:00,...,3.0,3-4级,南风,1,10,8,4,4,多云,8.0
4,XIY,SZX,HU7871,1451606700,1451616600,NaN,NaN,0.0,取消,2016-01-01 00:05:00+00:00,...,2.0,微风,无持续风向,1,10,10,4,5,雾,5.0


In [34]:
All_Data.rename(columns={'得分':'出发机场天气得分'})

,出发机场,到达机场,航班编号,计划起飞时间,计划到达时间,实际起飞时间,实际到达时间,飞机编号,航班是否取消,计划起飞时间1,...,到达机场aqiLevel,到达机场风力,到达机场风向,是否延误3小时以上,出发机场风力等级评分,到达机场风力等级评分,出发机场aqi等级评分,到达机场aqi等级评分,天气,出发机场天气得分
0,SWA,TAO,CZ3855,1451606400,1451619600,NaN,NaN,0.0,取消,2016-01-01 00:00:00+00:00,...,3.0,4-5级,西南风,1,10,7,4,4,多云,8.0
1,NNG,CAN,CZ3295,1451606400,1451610600,NaN,NaN,0.0,取消,2016-01-01 00:00:00+00:00,...,2.0,微风,无持续风向,1,10,10,5,5,多云,8.0
2,SWA,NGB,CZ3855,1451606400,1451610900,NaN,NaN,0.0,取消,2016-01-01 00:00:00+00:00,...,2.0,微风,东南风,1,10,10,4,5,多云,8.0
3,CSX,TNA,CZ3983,1451606400,1451613600,NaN,NaN,0.0,取消,2016-01-01 00:00:00+00:00,...,3.0,3-4级,南风,1,10,8,4,4,多云,8.0
4,XIY,SZX,HU7871,1451606700,1451616600,NaN,NaN,0.0,取消,2016-01-01 00:05:00+00:00,...,2.0,微风,无持续风向,1,10,10,4,5,雾,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5244219,TSN,DLC,NH8413,1494135300,1494139200,1.494137e+09,1.494140e+09,3283.0,正常,2017-05-07 05:35:00+00:00,...,NaN,NaN,NaN,0,0,0,0,0,NaN,NaN
5244220,SHE,DLC,CF9063,1495500000,1495504200,NaN,1.495502e+09,3284.0,正常,2017-05-23 00:40:00+00:00,...,NaN,NaN,NaN,1,0,0,0,0,NaN,NaN
5244221,NNG,WUH,EU2703,1494300300,1494306900,1.494301e+09,1.494307e+09,3285.0,正常,2017-05-09 03:25:00+00:00,...,NaN,NaN,NaN,0,0,0,0,0,NaN,NaN
5244222,KMG,TCZ,MU5762,1495965600,1495969500,1.495966e+09,1.495968e+09,3286.0,正常,2017-05-28 10:00:00+00:00,...,NaN,NaN,NaN,0,0,0,0,0,NaN,NaN


In [35]:
All_Data['到达机场天气'].unique()

array(['多云', '晴转多云', '晴', '雾转晴', '雾', '小雨转阵雨', nan, '阴', '阴转多云', '阵雪',
       '阴转小雨', '多云转阴', '小雨', '多云转晴', '小雪转阵雪', '小雪', '多云转小雪', '多云转小雨',
       '晴转阴', '晴转雾', '多云转阵雨', '阴转阵雨', '雾转阴', '小雨转晴', '雾转多云', '多云转雾',
       '小雨转阴', '阵雨', '小雨转中雨', '多云转中雨', '晴转小雪', '中雨转小雨', '小雨转多云', '中雨转阴',
       '中雨', '中雨转阵雨', '中雨转多云', '晴转阵雪', '小雪转晴', '阵雨转小雨', '小雪转雾', '雨夹雪转阴',
       '阵雪转多云', '阴转雾', '多云转中雪', '阵雪转阴', '小雪转多云', '多云转阵雪', '阵雨转阴',
       '小雨转雨夹雪', '多云转雨夹雪', '雨夹雪转多云', '阴转晴', '阴转小雪', '阴转中雨', '雾转小雪',
       '阴转雨夹雪', '小雨转大雨', '小雪转中雪', '雨夹雪转中雪', '阵雨转多云', '中雪转小雪', '中雪转阵雪',
       '阴转阵雪', '阴转中雪', '雨夹雪', '晴转小雨', '雨夹雪转小雨', '小雪转阴', '中雪', '晴转阵雨',
       '雨夹雪转大雪', '大雪', '大雪转中雪', '小雪转雨夹雪', '阵雨转小雪', '雨夹雪转小雪', '大雨',
       '大雨转小雨', '中雨转大雨', '阵雨转晴', '小雨转小雪', '小雨转中雪', '雷阵雨转多云', '小雨转雾', '浮尘',
       '阵雨转中雨', '雷阵雨转小雨', '阵雨转雷阵雨', '中雪转阴', '雾转中雨', '雨夹雪转晴', '大雪转小雪',
       '大雪转阴', '小雨转冻雨', '浮尘转晴', '雨夹雪转阵雪', '阵雪转晴', '阵雨转雾', '中雪转晴', '扬沙转阴',
       '扬沙', '雾转小雨', '雾转雷阵雨', '浮尘转多云', '扬沙转晴', '晴转浮尘', '雷阵雨转大雨', '多云转雷阵雨',
       '

In [36]:
len(All_Data['到达机场天气'].unique())

215

In [37]:
ArriWeatherScore = pd.read_excel('/Users/tuyu/000000毕业设计/ArrWeatherScore.xlsx')
All_Data = pd.merge(All_Data, ArriWeatherScore, how='left',left_on='到达机场天气', right_on='天气')
All_Data.rename(columns={'得分':'到达机场天气得分'})
All_Data.head()                                 

,出发机场,到达机场,航班编号,计划起飞时间,计划到达时间,实际起飞时间,实际到达时间,飞机编号,航班是否取消,计划起飞时间1,...,到达机场风向,是否延误3小时以上,出发机场风力等级评分,到达机场风力等级评分,出发机场aqi等级评分,到达机场aqi等级评分,天气_x,得分_x,天气_y,得分_y
0,SWA,TAO,CZ3855,1451606400,1451619600,NaN,NaN,0.0,取消,2016-01-01 00:00:00+00:00,...,西南风,1,10,7,4,4,多云,8.0,多云,8.0
1,NNG,CAN,CZ3295,1451606400,1451610600,NaN,NaN,0.0,取消,2016-01-01 00:00:00+00:00,...,无持续风向,1,10,10,5,5,多云,8.0,晴转多云,8.8
2,SWA,NGB,CZ3855,1451606400,1451610900,NaN,NaN,0.0,取消,2016-01-01 00:00:00+00:00,...,东南风,1,10,10,4,5,多云,8.0,晴转多云,8.8
3,CSX,TNA,CZ3983,1451606400,1451613600,NaN,NaN,0.0,取消,2016-01-01 00:00:00+00:00,...,南风,1,10,8,4,4,多云,8.0,晴,10.0
4,XIY,SZX,HU7871,1451606700,1451616600,NaN,NaN,0.0,取消,2016-01-01 00:05:00+00:00,...,无持续风向,1,10,10,4,5,雾,5.0,多云,8.0


In [38]:
All_Data = All_Data.rename(columns={'得分_x':'出发机场天气得分','得分_y':'到达机场天气得分' })
del(All_Data['天气_x'])
del(All_Data['天气_y'])

In [39]:
All_Data['出发机场最高气温'].replace(np.nan, 100, inplace=True)
All_Data['出发机场最高气温'].replace(np.inf, 100, inplace=True)
All_Data['出发机场最低气温'].replace(np.nan, -100, inplace=True)
All_Data['出发机场最低气温'].replace(np.inf, -100, inplace=True)

All_Data['出发机场最高气温'].astype('int')
All_Data['出发机场最低气温'].astype('int')

0           13
1           13
2           13
3            9
4           -1
          ... 
5244219   -100
5244220   -100
5244221   -100
5244222   -100
5244223   -100
Name: 出发机场最低气温, Length: 5244224, dtype: int64

In [40]:
print(sum(All_Data['出发机场最高气温'].isnull()),sum(All_Data['出发机场最低气温'].isnull()),
     (sum(All_Data['出发机场最高气温'].isnull() & All_Data['出发机场最低气温'].isnull())))

0 0 0


In [42]:
#出发机场计划起飞时刻温度
def Wendu(Gtemp, Ltemp, time):
    Gtemp = int(Gtemp)
    Ltemp = int(Ltemp)
    
    if 3 <= time <= 5: 
        temp = Ltemp 
    elif 6 <= time <= 8:
        temp = Ltemp + 4
    elif 9 <= time <= 10:
        temp = Gtemp -3
    elif 11 <= time <= 13:
        temp = Gtemp
    elif 14<= time <= 17:
        temp = Gtemp - 2
    elif 18 <= time <= 21:
        temp = Gtemp - 5
    else :
        temp = Ltemp + 2
        
    return temp

In [43]:
All_Data['出发机场起飞时刻温度'] = All_Data.apply(lambda x : Wendu(x['出发机场最高气温'], x['出发机场最低气温'], x['计划起飞时刻']), axis=1)


In [44]:
All_Data['到达机场最高气温'].replace(np.nan, 100, inplace=True)
All_Data['到达机场最高气温'].replace(np.inf, 100, inplace=True)
All_Data['到达机场最低气温'].replace(np.nan, -100, inplace=True)
All_Data['到达机场最低气温'].replace(np.inf, -100, inplace=True)

All_Data['到达机场最高气温'].astype('int')
All_Data['到达机场最低气温'].astype('int')

0            4
1           12
2            7
3            4
4           16
          ... 
5244219   -100
5244220   -100
5244221   -100
5244222   -100
5244223   -100
Name: 到达机场最低气温, Length: 5244224, dtype: int64

In [45]:
All_Data['到达机场起飞时刻温度'] = All_Data.apply(lambda x : Wendu(x['到达机场最高气温'], x['到达机场最低气温'], x['计划到达时刻']), axis=1)

In [46]:
#空数据用0来表示
All_Data['出发机场起飞时刻温度得分'] = All_Data['出发机场起飞时刻温度'].apply(lambda x : 5 if -5 <= x <= 40 else (4 if -15 <= x < -5 
                                                                    else (3 if -25 <= x < -15 else (4 if 40 < x <= 50 else 0))))

In [47]:
#空数据用0来表示
All_Data['到达机场起飞时刻温度得分'] = All_Data['到达机场起飞时刻温度'].apply(lambda x : 5 if -5 <= x <= 40 else (4 if -15 <= x < -5 
                                                                    else (3 if -25 <= x < -15 else (4 if 40 < x <= 50 else 0))))

In [48]:
Data = All_Data

In [49]:
#提取飞行日期 取值范围是1-31
Data['计划起飞日'] = Data['计划起飞时间1'].apply(lambda x: x[5:7] if not(pd.isnull(x)) else None)
Data['计划到达日'] = Data['计划到达时间1'].apply(lambda x: x[5:7] if not(pd.isnull(x)) else None)
Data['实际起飞日'] = Data['实际到达时间1'].apply(lambda x: x[5:7] if not(pd.isnull(x)) else None)
Data['实际到达日'] = Data['实际到达时间1'].apply(lambda x: x[5:7] if not(pd.isnull(x)) else None)

In [50]:
import datetime
#提取飞行星期日，取值范围是1-7
Data['计划起飞星期日'] = Data['计划起飞日期'].apply(lambda x :datetime.datetime.strptime(x, "%Y-%m-%d").weekday() + 1)#提取飞行星期日，取值范围是1-7
Data['计划到达星期日'] = Data['计划到达日期'].apply(lambda x :datetime.datetime.strptime(x, "%Y-%m-%d").weekday() + 1)

In [51]:
Data['实际起飞时间1'] = pd.to_datetime(Data['实际起飞时间'], unit = 's', utc = True)
Data['实际到达时间1'] = pd.to_datetime(Data['实际到达时间'], unit = 's', utc = True)
Data['实际起飞日期'] = Data['实际起飞时间1'].apply(lambda x:x.strftime('%Y-%m-%d') if not(pd.isnull(x)) else None)
Data['实际到达日期'] = Data['实际到达时间1'].apply(lambda x:x.strftime('%Y-%m-%d') if not(pd.isnull(x)) else None)


In [52]:
#提取实际起飞和到达星期日，取值范围是1-7，若为空值，则设为None
Data['实际起飞星期日'] = Data['实际起飞日期'].apply(lambda x :datetime.datetime.strptime(x, "%Y-%m-%d").weekday() + 1  if not(pd.isnull(x)) else None)

Data['实际到达星期日'] = Data['实际到达日期'].apply(lambda x :datetime.datetime.strptime(x, "%Y-%m-%d").weekday() + 1  if not(pd.isnull(x)) else None)


In [53]:
special2016 = pd.read_table('/Users/tuyu/000000毕业设计/Graduation_project_python/2016Holiday.csv', sep=',',encoding='utf-8',low_memory=False)
special2017 = pd.read_table('/Users/tuyu/000000毕业设计/Graduation_project_python/2017Holiday.csv', sep=',',encoding='utf-8',low_memory=False)
specialDate = pd.concat([special2016, special2017], axis=0, ignore_index=True)

In [54]:
#是否为节假日，工作日为1 ，休假日为0
Data = pd.merge(Data, specialDate, how='left',left_on='计划起飞日期', right_on='Date')

In [55]:
def isWorkDay(tag, date):
    date = int(date)
    if tag == '休假':
        ans = 0
    elif tag == '补班':
        ans = 1
    elif 1<= date <= 5:
        ans = 1      
    else:
        ans = 0
    return ans

In [56]:
Data['计划起飞星期日'] = Data['计划起飞星期日'].apply(int)
Data['计划起飞日期是否为工作日'] = Data.apply(lambda x  : isWorkDay(x['Tag'], x['计划起飞星期日']), axis=1)
Data['计划到达星期日'] = Data['计划到达星期日'].apply(int)
Data['计划到达日期是否为工作日'] = Data.apply(lambda x  :  isWorkDay(x['Tag'], x['计划到达星期日']), axis=1)
# Data.apply(lambda x  : isWorkDay( x['Tag'],  x['计划到达星期日']), axis=1)
Data['实际起飞日期是否为工作日'] = Data.apply(lambda x  :  isWorkDay( x['Tag'],  x['实际起飞星期日']) if not(pd.isnull(x['实际起飞星期日'])) else None , axis=1)
Data['实际到达日期是否为工作日'] = Data.apply(lambda x  : isWorkDay( x['Tag'],  x['实际到达星期日'])if not(pd.isnull(x['实际到达星期日'])) else None, axis=1)

In [57]:
Data

,出发机场,到达机场,航班编号,计划起飞时间,计划到达时间,实际起飞时间,实际到达时间,飞机编号,航班是否取消,计划起飞时间1,...,实际到达日期,实际起飞星期日,实际到达星期日,Date,Holiday,Tag,计划起飞日期是否为工作日,计划到达日期是否为工作日,实际起飞日期是否为工作日,实际到达日期是否为工作日
0,SWA,TAO,CZ3855,1451606400,1451619600,NaN,NaN,0.0,取消,2016-01-01 00:00:00+00:00,...,None,NaN,NaN,2016-01-01,元旦,休假,0,0,NaN,NaN
1,NNG,CAN,CZ3295,1451606400,1451610600,NaN,NaN,0.0,取消,2016-01-01 00:00:00+00:00,...,None,NaN,NaN,2016-01-01,元旦,休假,0,0,NaN,NaN
2,SWA,NGB,CZ3855,1451606400,1451610900,NaN,NaN,0.0,取消,2016-01-01 00:00:00+00:00,...,None,NaN,NaN,2016-01-01,元旦,休假,0,0,NaN,NaN
3,CSX,TNA,CZ3983,1451606400,1451613600,NaN,NaN,0.0,取消,2016-01-01 00:00:00+00:00,...,None,NaN,NaN,2016-01-01,元旦,休假,0,0,NaN,NaN
4,XIY,SZX,HU7871,1451606700,1451616600,NaN,NaN,0.0,取消,2016-01-01 00:05:00+00:00,...,None,NaN,NaN,2016-01-01,元旦,休假,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5244219,TSN,DLC,NH8413,1494135300,1494139200,1.494137e+09,1.494140e+09,3283.0,正常,2017-05-07 05:35:00+00:00,...,2017-05-07,7.0,7.0,2017-05-07,十二,休假,0,0,0.0,0.0
5244220,SHE,DLC,CF9063,1495500000,1495504200,NaN,1.495502e+09,3284.0,正常,2017-05-23 00:40:00+00:00,...,2017-05-23,NaN,2.0,NaN,NaN,NaN,1,1,NaN,1.0
5244221,NNG,WUH,EU2703,1494300300,1494306900,1.494301e+09,1.494307e+09,3285.0,正常,2017-05-09 03:25:00+00:00,...,2017-05-09,2.0,2.0,NaN,NaN,NaN,1,1,1.0,1.0
5244222,KMG,TCZ,MU5762,1495965600,1495969500,1.495966e+09,1.495968e+09,3286.0,正常,2017-05-28 10:00:00+00:00,...,2017-05-28,7.0,7.0,2017-05-28,初三,休假,0,0,0.0,0.0


In [ ]:
Data.to_csv(r"/Users/tuyu/000000毕业设计/Flight_Delay/DataTotal.csv", index=None, encoding="gb2312")